# Loading Trained Model

In [ ]:
if DATASET == "ImageNet2012":
    model = MyResNet34(num_classes=COUNT_OF_CLASSES).to(device)
else:
    model = MyResNet_CIFAR(num_classes=COUNT_OF_CLASSES, num_layer_factor=5).to(device)


model.load_state_dict(torch.load(f"models/Myresnet32{DATASET}.pth"))

In [ ]:
model.named_modules

In [ ]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_loss /= len(test_dataloader)
test_acc = correct / total

In [ ]:
print(test_acc)